In [31]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px  
import plotly.io as pio
pio.renderers.default = "notebook_connected"


pd.options.mode.chained_assignment = None  # default='warn'


In [32]:
csv = 'all.csv'
##leer el csv
dfAll = pd.read_csv(csv)
##volver las columnas date and release_date en objetos de fecha
dfAll['date'] = pd.to_datetime(dfAll['date'], format='%d/%m/%Y')
dfAll['release_date'] = pd.to_datetime(dfAll['release_date'], format='%d/%m/%Y')
##Quitar los duplicados
dfAll = dfAll.groupby(['artist_full', 'song', 'date'])[['rank', 'main_artist', 'featured_artist_1', 'featured_artist_2',
'featured_artist_3', 'semanas_en_rank', 'peak_rank', 'explicit', 'release_date', 'cantidad_de_features', 'duration_ms']].first().reset_index()
##Separar los años
dfAll['year'] = dfAll.date.dt.year
dfAll['release_year'] = dfAll.release_date.dt.year
##Columa con canción completa
dfAll['canción'] = dfAll['song'] + '  ' + '<br>' + dfAll['artist_full'] + '  '
##Boolean si tiene feat,
dfAll['has_features'] = (dfAll['featured_artist_1'].ne('') & dfAll['featured_artist_1'].notnull()).astype(int)
dfAll['month'] = dfAll.release_date.dt.month_name()
##seleccionar solo los años 2010 y 2020
dfAll= dfAll[(dfAll['year'] == 2010) | (dfAll['year'] == 2020)]


In [33]:
##contar todas las entradas explicitas y no explicitas
dfExplicit = dfAll.groupby(['explicit', 'year'])['explicit'].count().reset_index(name="cantidad")
dfExplicit= dfExplicit.sort_values(by ="year")


In [34]:
##Sunburst con los porcentajes generales de canciones explícitas
figExplicitAll = sunburstYearExplicit = px.sunburst(dfExplicit, title ='¿Es explícita?',
                                        path=['explicit', 'year'],
                                        values = 'cantidad',
                                        color = 'cantidad',
                                        labels = {'parent' : '','explicit': 'explícito', 'id' : 'Sección', 'labels' : 'Sección', 'cantidad_sum' : 'Total de entradas'}
                                        )
figExplicitAll.update_traces(textinfo="label+percent root", insidetextorientation='horizontal' )
                                        
figExplicitAll.show("notebook_connected")








In [35]:
##Separar la cantidad de explícitos por año
dataFrameExplicit = dfAll[dfAll['explicit'] == 'si'].groupby('year')['explicit'].count().reset_index()
dataFrameExplicit = dataFrameExplicit.rename(columns={'explicit' : 'si'})

##Separar la cantidad de NO explícitos por año
dataFrameNotExplicit = dfAll[dfAll['explicit'] == 'no'].groupby('year')['explicit'].count().reset_index()
dataFrameNotExplicit = dataFrameNotExplicit.rename(columns={'explicit' : 'no'})

##Unir en cuadro comparativo
dataFrameExplicitBarChart = dataFrameExplicit.merge(dataFrameNotExplicit, how='inner', on='year')

##Suma de entradas totales por año + creación de columna de porcentaje + fusión con el frame por entradas
dataFrameExplicitBarChart['total'] = dataFrameExplicitBarChart['si'] + dataFrameExplicitBarChart['no']
dfExplicitComb = dfExplicit.merge(dataFrameExplicitBarChart, how='inner', on='year')
dfExplicitComb['percentage'] = (dfExplicitComb['cantidad']) / (dfExplicitComb['total'] / 100)
y = dfExplicitComb['percentage']
##figura con anotación porcentaje canciones explícitas
figExplicitAllBars = px.bar(dfExplicitComb, 
                                    x=['year', 'explicit'],
                                    y = 'percentage',                               
                                    labels = {'explicit' : '¿Es explícita?', 'percentage' : 'Porcentaje', 'year' : 'Año', 'value' : 'Año'},
                                    title = 'Porcentaje de canciones explícitas por año',
                                    color= 'explicit',
                                    
                                    )
figExplicitAllBars.update_xaxes(type='category')
figExplicitAllBars.update_xaxes(fixedrange=True)
figExplicitAllBars.update_yaxes(fixedrange=True)

figExplicitAllBars.add_annotation(text="75.55%",
                  xref="paper", yref="paper",
                  x=0.22, y=0.4, showarrow=False)

figExplicitAllBars.add_annotation(text="24.45%",
                  xref="paper", yref="paper",
                  x=0.22, y=0.9, showarrow=False)

figExplicitAllBars.add_annotation(text="57.74%",
                  xref="paper", yref="paper",
                  x=0.78, y=0.4, showarrow=False)

figExplicitAllBars.add_annotation(text="42.26%",
                  xref="paper", yref="paper",
                  x=0.78, y=0.9, showarrow=False)




figExplicitAllBars.show()





In [36]:
## pie chart de entradas explícitas, decada 2010s                                       
figExplicit10s = pieChartExplicit10s = px.pie(dfExplicit[dfExplicit.year < 2012], 
                        title= 'Canciones explícitas, 2010',
                            values='cantidad',
                            names = 'explicit',
                            labels= {'explicit' : 'Explícita', 'cantidad' : 'Total de entradas'})
figExplicit10s.show()


## pie chart de entradas explícitas, decada 2020s         
figExplicit20s = pieChartExplicit20s = px.pie(dfExplicit[dfExplicit.year > 2012], 
                        title= 'Canciones explícitas, 2020',
                            values='cantidad',
                            names = 'explicit',
                            labels= {'explicit' : 'Explícita', 'cantidad' : 'Total de entradas'})

In [37]:
## Lista de meses
meses = {'month' : ['January', 'February', 'March', 'April','May','June','July', 'August', 'September', 'October', 'November', 'December'],
'Mes' : ['Enero', 'Febrero', 'Marzo', 'Abril','Mayo','Junio','Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']}
dfMeses = pd.DataFrame(data=meses)
#Juntar con meses
songsPerYeartodo = dfAll.merge(dfMeses, how= 'left', on= 'month')

songsPerYeartodo = songsPerYeartodo.sort_values(by="rank", ascending = True)
##Agrupar canciones y rankearlas

songsPerYeartodo = songsPerYeartodo.groupby(['canción', 'artist_full', 'date'])[['rank','peak_rank','release_year', 'song', 'year', 'Mes']].first().reset_index()
songsPerYeartodo = songsPerYeartodo.sort_values(by='year', ascending = True)

##seleccionar solo los que llegaron al n1
songsPerYearN1 = songsPerYeartodo[songsPerYeartodo['rank'] == 1]

#agrupar por canción
songsPerYear = songsPerYearN1.groupby(['year', 'canción', 'artist_full']).agg({'rank': 'max', 
                                                            'peak_rank' : 'min', 
                                                            'release_year' : 'first', 'Mes' : 'first',
                                                            'song' : 'count'}).rename(columns={'rank':'rank_máximo',                      
                                                                                                'song' : 'semanas_totales'}).reset_index()

#agrupar por canción
songsPerYeartodo = songsPerYeartodo.groupby(['year', 'canción', 'artist_full']).agg({'rank': 'max', 
                                                            'peak_rank' : 'min', 
                                                            'release_year' : 'first', 'Mes' : 'first',
                                                            'song' : 'count'}).rename(columns={'rank':'rank_máximo',                      
                                                                                                'song' : 'semanas_totales'}).reset_index() 


##Orden de prioridad del ranking
songsPerYear = songsPerYear.sort_values(by=['year', 'peak_rank', 'semanas_totales'], ascending=[True, True, False]).reset_index()



#gráfico mostrando las canciones en el n1 con más semanas en el 2010
TopSongs2010 = TopSongsChart = px.bar(songsPerYear[songsPerYear['year'] == 2010][::-1],
                                    x='semanas_totales',
                                    y = 'canción',
                                    height = 900,
                                    hover_data=['peak_rank', 'semanas_totales', 'release_year', 'Mes'],
                                    labels = {'rank_máximo' : 'Rank máximo', 'canción' : 'Canción',
                                    'year' : 'Año', 'semanas_totales' : 'Semanas en el rank', 'peak_rank' : 'Mejor puesto', 'release_year' : 'Año de lanzamiento'},
                                    title = 'Top canciones del año 2010, ordenadas cantidad de semanas en el chart',                                   
                                    range_x=[0,11] )
                                    

                             
TopSongs2010.show()


#gráfico mostrando las canciones en el n1 con más semanas en el 2020
TopSongs2020 = TopSongsChart2020 = px.bar(songsPerYear[songsPerYear['year'] == 2020][::-1],
                                    x='semanas_totales',
                                    y = 'canción',
                                    hover_data=['peak_rank', 'semanas_totales', 'release_year', 'Mes'],
                                    labels = {'rank_máximo' : 'Rank máximo', 'canción' : 'Canción',
                                    'year' : 'Año', 'semanas_totales' : 'Semanas en el rank', 'peak_rank' : 'Mejor puesto', 'release_year' : 'Año de lanzamiento'},
                                    title = 'Top canciones del año 2020, ordenadas por cantidad de semanas en el chart',                                   
                                    height=900,
                                    range_x=[0,11]
                                    
                                    )
  
TopSongs2020.show()



In [38]:
#separar por decada para futuro

todo2010s = dfAll[dfAll.year < 2012]
todo2020s = dfAll[dfAll.year >2012]

#seleccionar canciones que llegaron a n1, 2010s

songsThatReachedN12010s = songsPerYear[songsPerYear['year']== 2010]

#seleccionar canciones que NO llegaron a n1, 2010s
totalSongs2010s = songsPerYeartodo[songsPerYeartodo['year']== 2010]

#seleccionar canciones que llegaron a n1, 2020s
songsThatReachedN12020s = songsPerYear[songsPerYear['year']== 2020]

#seleccionar canciones que NO llegaron a n1, 2010s
totalSongs2020s = songsPerYeartodo[songsPerYeartodo['year']== 2020]


#Figuras con canciones que llegaron a n1

figN1PercentagePie2010s = px.pie(
                                title = 'Porcentaje de canciones que llegaron a N°1, 2010',
                                names = ['No llegaron', 'Llegaron a N°1'], 
                                values = [(totalSongs2010s.shape[0] - songsThatReachedN12010s.shape[0]), songsThatReachedN12010s.shape[0]])
figN1PercentagePie2010s.show()


figN1PercentagePie2020s = px.pie(
                                title = 'Porcentaje de canciones que llegaron a N°1, 2020',
                                names = ['No llegaron', 'Llegaron a N°1'], 
                                 values = [(totalSongs2020s.shape[0] - songsThatReachedN12020s.shape[0]), songsThatReachedN12020s.shape[0]])
figN1PercentagePie2020s.show()




In [39]:

#agrupar artistas y ordenarlos por cantidad de semanas
topMainArtists = dfAll.groupby(['canción', 'date'])[['release_year', 'song', 'year', 'main_artist']].first().reset_index()
topMainArtists = topMainArtists.groupby(['main_artist' , 'canción']).agg({'release_year' : 'first', 'year' : 'first', 
 'song' : 'count'}).rename(columns={'song' : 'semanas_totales'}).reset_index()
topMainArtists = topMainArtists.sort_values(by='semanas_totales', ascending = False)
topMainArtistsGroupedAll = topMainArtists.groupby('main_artist').agg({'canción' : 'count',
 'semanas_totales' : 'sum', 'year' : 'first'}).reset_index()

#artistas ordenados por cantidad de canciones
topMainArtistsGroupedAllMostSongs = topMainArtistsGroupedAll.sort_values(by='canción', ascending = False, ignore_index = True).head(10)
#artistas ordenados por cantidad de semanas
topMainArtistsGroupedAllSort = topMainArtistsGroupedAll.sort_values(by='semanas_totales', ascending = False, ignore_index = True).head(10)

#grafico artistas por semanas

figTopArtistsAll = px.bar(topMainArtistsGroupedAllSort.sort_values(by='semanas_totales', ascending = True ), 
                                                    x='main_artist', 
                                                    y='semanas_totales', 
                                                    color='year', 
                                                    labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año'},
                                                    hover_name="semanas_totales", 
                                                    title = 'Artistas con más semanas en el rank, 2010 + 2020',
                                                 
                                                    )
figTopArtistsAll.show()

#grafico artistas por canciones
figTopArtistsAllMostSongs = px.bar(topMainArtistsGroupedAllMostSongs.sort_values(by='canción', ascending = True ), 
                                                    x='main_artist', 
                                                    y='canción', 
                                                    color='year', 
                                                    labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año', 'canción' : 'Cantidad de canciones'},                                                
                                                    hover_name="year", 
                                                    title = 'Artistas con más canciones en el rank, 2010 + 2020',
                                   
                                                    )
figTopArtistsAllMostSongs.show()


In [40]:
##separar artistas por año
topMainArtists2010s = topMainArtists[(topMainArtists['year']== 2010)]
topMainArtists2020s = topMainArtists[(topMainArtists['year']==2020)]


##agrupados artistas
topMainArtistsGrouped2010s = topMainArtists2010s.groupby('main_artist').agg({'canción' : 'count',
 'semanas_totales' : 'sum', 'year' : 'first'}).reset_index()

##ordenar por cantidad de semanas
topMainArtistsGrouped2010s = topMainArtistsGrouped2010s.sort_values(by='semanas_totales', ascending = False)
Artists2010 = topMainArtistsGrouped2010s

##gráfico
topMainArtistsGrouped2010s['main_artist'] = topMainArtistsGrouped2010s['main_artist'] + '  '
topMainArtistsGrouped2010MostSongs = topMainArtistsGrouped2010s.sort_values(by='canción', ascending = False, ignore_index = True)
topMainArtistsGrouped2010MostSongs = topMainArtistsGrouped2010MostSongs[topMainArtistsGrouped2010MostSongs['canción'] > 1]

figTopArtists2010s = px.bar(topMainArtistsGrouped2010s.head(10).sort_values(by='semanas_totales', ascending = True), 
                                                    y='main_artist', 
                                                    x='semanas_totales', 
                                                    color='canción', 
                                                     labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año', 'canción' : 'Cantidad de canciones'},  
                                                    hover_name="semanas_totales", 
                                                    title = 'Artistas con más semanas en el rank 2010',
                                                   
                                                    )
figTopArtists2010s.show()

figTopArtists2010MostSongs = px.bar(topMainArtistsGrouped2010MostSongs.sort_values(by='canción', ascending = True ), 
                                                    y='main_artist', 
                                                    x='canción', 
                                                    color='semanas_totales',                                                 
                                                    hover_name="year", 
                                                    title = 'Artistas con más canciones en el rank, 2010',
                                                    labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año', 'canción' : 'Cantidad de canciones'},  
                                   
                                                    )
figTopArtists2010MostSongs.show()


In [41]:
##agrupar attistas 2020
topMainArtistsGrouped2020s = topMainArtists2020s.groupby('main_artist').agg({'canción' : 'count', 
'semanas_totales' : 'sum', 'year' : 'first'}).reset_index()

##ordenar por semanas 
topMainArtistsGrouped2020s = topMainArtistsGrouped2020s.sort_values(by='semanas_totales', ascending = False)
Artists2020 = topMainArtistsGrouped2020s

#gráfico
topMainArtistsGrouped2020s['main_artist'] = topMainArtistsGrouped2020s['main_artist'] + '  '
topMainArtistsGrouped2020MostSongs = topMainArtistsGrouped2020s.sort_values(by='canción', ascending = False, ignore_index = True)
topMainArtistsGrouped2020MostSongs = topMainArtistsGrouped2020MostSongs[topMainArtistsGrouped2020MostSongs['canción'] > 1]



figTopArtists2020s = px.bar(topMainArtistsGrouped2020s.head(10).sort_values(by='semanas_totales', ascending = True), 
                                y='main_artist', 
                                x='semanas_totales', 
                                color='canción', 
                                labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año', 'canción' : 'Cantidad de canciones'},  
                                hover_name="semanas_totales", 
                                title = 'Artistas con más semanas en el rank, 2020',
                                
                               )
figTopArtists2020s.show()


figTopArtists2020MostSongs = px.bar(topMainArtistsGrouped2020MostSongs.sort_values(by='canción', ascending = True ), 
                                                    y='main_artist', 
                                                    x='canción', 
                                                    color='semanas_totales',                                                 
                                                    hover_name="year", 
                                                    title = 'Artistas con más canciones en el rank, 2020',
                                                    labels = {'main_artist' : 'Artista', 'semanas_totales' : 'Semanas totales',
                                                    'year' : 'Año', 'canción' : 'Cantidad de canciones'},  
                                   
                                                    )
figTopArtists2020MostSongs.show()



In [42]:
dfNumOne2010s = songsThatReachedN12010s

dfNumOne2020s = songsThatReachedN12020s

#canciones en number 1
cancionesNumOne2010s = songsThatReachedN12010s['canción'].count()
cancionesNumOne2020s = songsThatReachedN12020s['canción'].count()


#artistas en number 1

dfNumOneArtistas2010s = songsThatReachedN12010s.groupby(['artist_full'])['year'].min().reset_index()
dfNumOneArtistas2020s = songsThatReachedN12020s.groupby(['artist_full'])['year'].min().reset_index()

ArtistasNumOne2010s = dfNumOneArtistas2010s.shape[0]
ArtistasNumOne2020s = dfNumOneArtistas2020s.shape[0]


#Cantidad de artistas unicos

promedioArtistas2010s = topMainArtistsGrouped2010s['main_artist'].count()
promedioArtistas2020s = topMainArtistsGrouped2020s['main_artist'].count()

#Cantidad de canciones únicas
cantidadDeCanciones2010s = topMainArtistsGrouped2010s['canción'].sum()
cantidadDeCanciones2020s = topMainArtistsGrouped2020s['canción'].sum()


topMainArtistsGroupedAllSongs = topMainArtists.groupby(['main_artist', 'year']).agg({'canción' : 'count',
 'semanas_totales' : 'sum'}).reset_index()
HCancionesPorArtista2010s = px.box(topMainArtistsGroupedAllSongs, 
                                            y='canción', 
                                            x='year', points="all", 
                                            title = 'Cantidad de canciones por artista en el ranking',
                                            labels = {'canción' : 'Cantidad de canciones', 'year' : 'Año', 'median' : 'Mediana'},
                                            color = 'year')
HCancionesPorArtista2010s.show()



#Diccionario para gáfico
dec = { 'Década' : ['2010s','2020s'], 
'Número de canciones distintas' : [cantidadDeCanciones2010s, cantidadDeCanciones2020s],
'Cantidad de artistas' : [promedioArtistas2010s, promedioArtistas2020s],
'Canciones que llegaron al número 1' : [cancionesNumOne2010s, cancionesNumOne2020s],
'Artistas que llegaron al número 1' : [ArtistasNumOne2010s,ArtistasNumOne2020s]
}


#Pasarlo a DataFrame
resumen = pd.DataFrame(data=dec)



#Gráfico


In [43]:
#canciones por año
canciones2010 = dfAll[dfAll['year'] == 2010].groupby(['canción']).count().shape[0]
canciones2020 = dfAll[dfAll['year'] == 2020].groupby('canción').count().shape[0]

#artistas por año
artistas2010 = dfAll[dfAll['year'] == 2010].groupby('main_artist').count().shape[0]
artistas2020 = dfAll[dfAll['year'] == 2020].groupby('main_artist').count().shape[0]

#columnas para gráfico
years = [2010,2020]
indicadores = ['canciones', 'artistas']

#gráfico cantidad de canciones y artistas
fig = go.Figure(data=[
    go.Bar(name='Cantidad de canciones', x=years, y=[canciones2010, canciones2020]),
    go.Bar(name='Cantidad de artistas', x=years, y=[artistas2010, artistas2020])
])
fig.update_layout(barmode='group')
fig.update_layout(title='Comparación: cantidad de artistas y canciones únicas en el ránking, 2010 vs 2020')

fig.show()

In [44]:
#contar semanas por canción
dfWeeks = dfAll.groupby(['canción', 'year'])['song'].count().reset_index()

#figura de semanas por canción
figSemanasPromedio = px.box(dfWeeks,
                            x='year', 
                            y='song', 
                            labels = {'year' : 'Año' , 'song' : 'Semanas'},
                            color='year',
                            points = 'all',
                            title = 'Cantidad de semanas en el ránking por canción, general')
figSemanasPromedio.show()

#contar semanas en el rank por artista
dfWeeksByArtist = dfAll.groupby(['main_artist', 'date'])[['song', 'year']].first().reset_index()
dfWeeksByArtist = dfWeeksByArtist.groupby(['main_artist', 'year'])['song'].count().reset_index()

#gráfico de semanas por artista
figSemanasPromedioPorArtista = px.box(dfWeeksByArtist,
                            x='year', 
                            y='song', 
                            labels = {'year' : 'Año' , 'song' : 'Semanas'},
                            color='year',
                            points = 'all',
 
                           title = 'Cantidad de semanas en el ránking por artista, general')
figSemanasPromedioPorArtista.show()



In [45]:
#seleccionar todas las canciones con features
featuredSongs = dfAll[dfAll['has_features'] == 1].groupby('canción')['year', 'main_artist' , 'featured_artist_1', 'featured_artist_2', 'featured_artist_3', 'cantidad_de_features'].first().reset_index()

##agrupar por año y cantidad de artistas invitados
featuredSongsCantidad = featuredSongs.groupby(['cantidad_de_features', 'year'])['canción'].count().reset_index()
featuredSongsCantidad12010 = featuredSongs[(featuredSongs['year'] == 2010) & (featuredSongs['cantidad_de_features'] == 1)].shape[0]
featuredSongsCantidad12020 = featuredSongs[(featuredSongs['year'] == 2020) & (featuredSongs['cantidad_de_features'] == 1)].shape[0]

featuredSongsCantidad22010 = featuredSongs[(featuredSongs['year'] == 2010) & (featuredSongs['cantidad_de_features'] == 2)].shape[0]
featuredSongsCantidad22020 = featuredSongs[(featuredSongs['year'] == 2020) & (featuredSongs['cantidad_de_features'] == 2)].shape[0]

featuredSongsCantidad32010 = featuredSongs[(featuredSongs['year'] == 2010) & (featuredSongs['cantidad_de_features'] == 3)].shape[0]
featuredSongsCantidad32020 = featuredSongs[(featuredSongs['year'] == 2020) & (featuredSongs['cantidad_de_features'] == 3)].shape[0]

#gráfico comparando cantidad de artsitas invitados por año
figFeat = go.Figure(data=[
    go.Bar(name='1 artista invitado', x=years, y=[featuredSongsCantidad12010, featuredSongsCantidad12020], text=[featuredSongsCantidad12010, featuredSongsCantidad12020], textposition='auto',),
    go.Bar(name='2 artistas invitados', x=years, y=[featuredSongsCantidad22010, featuredSongsCantidad22020], text=[featuredSongsCantidad22010, featuredSongsCantidad22020], textposition='auto',),
    go.Bar(name='3 artistas invitados', x=years, y=[featuredSongsCantidad32010, featuredSongsCantidad32020], text=[featuredSongsCantidad32010, featuredSongsCantidad32020], textposition='auto',),
    
])
figFeat.update_layout(barmode='group')
figFeat.update_layout(title='Cantidad de canciones con 1, 2 o 3 artistas invitados, 2010 vs 2020')

figFeat.show()

#Filtrar 2020
featured2020 = featuredSongs[featuredSongs['year'] > 2012]
featured2020Num = featured2020.shape[0]

#Filtrar 2010
featured2010 = featuredSongs[featuredSongs['year'] < 2012]
featured2010Num = featured2010.shape[0]

#seleccionar todas las otras canciones
totalsongs = dfAll[dfAll['has_features'] == 0]
totalsongs = totalsongs.groupby(['canción'])['year'].first().reset_index()

#Filtrar 2020
totalsongs2020 = totalsongs[totalsongs['year'] > 2012]
totalsongs2020Num = totalsongs2020.shape[0]

#Filtrar 2010
totalsongs2010 = totalsongs[totalsongs['year'] < 2012]
totalsongs2010Num = totalsongs2010.shape[0]

#diccionario con datos, 2020s y 2010s

dPieChartFeatures2010s = {'cantidad': [totalsongs2010Num, featured2010Num], 
                                    '¿Tiene artistas invitados?' : ['no', 'si']}


dPieChartFeatures2020s = {'cantidad': [totalsongs2020Num, featured2020Num], 
                                    '¿Tiene artistas invitados?' : ['no', 'si']}


#Converir diccionario a DataFrame
dfPieChartFeat2010s = pd.DataFrame(data=dPieChartFeatures2010s)
dfPieChartFeat2020s = pd.DataFrame(data=dPieChartFeatures2020s)

# gráficos porcentaje de canciones con artistas invitados
figpercentagefeat2020 = px.pie(dfPieChartFeat2020s, 
                                names= '¿Tiene artistas invitados?', 
                                values='cantidad',
                                title='Porcentaje de canciones con artistas invitados, 2020s')

figpercentagefeat2020.show()

figpercentagefeat2010 = px.pie(dfPieChartFeat2010s, 
                                names= '¿Tiene artistas invitados?', 
                                values='cantidad',
                                title='Porcentaje de canciones con artistas invitados, 2010s')

figpercentagefeat2010.show()




C:\Users\MARIAT~1\AppData\Local\Temp/ipykernel_66116/626250176.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [46]:
## Lista de artistas invitados 2020

FeaturedArtists2020List = featured2020[['featured_artist_1', 'featured_artist_2','featured_artist_3']]
FeaturedArtists2020List = FeaturedArtists2020List['featured_artist_1'].tolist() + FeaturedArtists2020List['featured_artist_2'].tolist() + FeaturedArtists2020List['featured_artist_3'].tolist()
feat2020Data = {'Artistas invitados' : FeaturedArtists2020List,
                'Conteo' : FeaturedArtists2020List }
FeaturedArtists2020 = pd.DataFrame(data=feat2020Data)
FeaturedArtists2020 = FeaturedArtists2020.groupby('Artistas invitados')['Conteo'].count().reset_index()
FeaturedArtists2020 = FeaturedArtists2020.sort_values(by='Conteo', ascending = False)



## Lista de artistas invitados 2010
FeaturedArtists2010List = featured2010[['featured_artist_1', 'featured_artist_2','featured_artist_3']]
FeaturedArtists2010List = FeaturedArtists2010List['featured_artist_1'].tolist() + FeaturedArtists2010List['featured_artist_2'].tolist() + FeaturedArtists2010List['featured_artist_3'].tolist()
feat2010Data = {'Artistas invitados' : FeaturedArtists2010List, 
                'Conteo' : FeaturedArtists2010List}
FeaturedArtists2010 = pd.DataFrame(data=feat2010Data)
FeaturedArtists2010 = FeaturedArtists2010.groupby('Artistas invitados')['Conteo'].count().reset_index()
FeaturedArtists2010 = FeaturedArtists2010.sort_values(by='Conteo', ascending = False)


#tabla mostrando artistas invitados 2020
FeaturedArtists2020Table = go.Figure(data=[go.Table(
    header=dict(values=list(FeaturedArtists2020.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[FeaturedArtists2020['Artistas invitados'], FeaturedArtists2020['Conteo']],
               fill_color='lavender',
               align='left'))
])
FeaturedArtists2020Table.update_layout(title='Artistas invitados más frecuentes, 2020')

FeaturedArtists2020Table.show()

#tabla mostrando artistas invitados 2010




FeaturedArtists2010Table = go.Figure(data=[go.Table(
    header=dict(values=list(FeaturedArtists2010.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[FeaturedArtists2010['Artistas invitados'], FeaturedArtists2010['Conteo']],
               fill_color='lavender',
               align='left'))
])
FeaturedArtists2010Table.update_layout(title='Artistas invitados más frecuentes, 2010')

FeaturedArtists2010Table.show()




In [47]:
## featured astists que aparecen en las dos décadas

##unir dos listas
bothdecadesFull = FeaturedArtists2020.merge(FeaturedArtists2010, how='outer', on='Artistas invitados')
BothDecades  = FeaturedArtists2020.merge(FeaturedArtists2010, how='inner', on='Artistas invitados')

#sumar conteos
BothDecades['Conteo'] = BothDecades['Conteo_x'] + BothDecades['Conteo_y']
BothDecades = BothDecades.drop(['Conteo_x', 'Conteo_y'], axis=1)
BothDecades = BothDecades.sort_values(by='Conteo', ascending = False)

#tabla mostrando los artistas invitados de las dos decadas
FeaturedArtistsBothDecadesTable = go.Figure(data=[go.Table(
    header=dict(values=list(BothDecades.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[BothDecades['Artistas invitados'], BothDecades['Conteo']],
               fill_color='lavender',
               align='left'))
])
FeaturedArtistsBothDecadesTable.update_layout(title='Artistas invitados más frecuentes en ambas décadas')


In [48]:
#Artistas principales de las dos decadas

Artists2020 = topMainArtistsGrouped2020s

Artists2010 = topMainArtistsGrouped2010s

MainArtists2020 = Artists2020

MainArtists2010 = Artists2010

#unir los dos años
AllMainArtists = Artists2020.merge(Artists2010, how='outer', on='main_artist')
AllArtists  = bothdecadesFull.merge(AllMainArtists, how='outer', left_on='Artistas invitados', right_on='main_artist')

BothDecadesArtists  = MainArtists2020.merge(MainArtists2010, how='inner', on='main_artist')

#sumar la cantidad de canciones por artista
BothDecadesArtists['Conteo'] = BothDecadesArtists['canción_x'] + BothDecadesArtists['canción_y']
BothDecadesArtists = BothDecadesArtists.drop(['canción_x', 'semanas_totales_x', 'semanas_totales_y', 'year_x', 'year_y', 'canción_y'], axis=1)
BothDecadesArtists = BothDecadesArtists.rename(columns=({'main_artist' : 'Artista principal'}))
BothDecadesArtists = BothDecadesArtists.sort_values(by='Conteo', ascending = False)

#tabla con artistas en las dos decadas
ArtistsBothDecadesTable = go.Figure(data=[go.Table(
    header=dict(values=list(BothDecadesArtists.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[BothDecadesArtists['Artista principal'], BothDecadesArtists['Conteo']],
               fill_color='lavender',
               align='left'))
])

ArtistsBothDecadesTable.show()



In [49]:

##Primera fecha que llegaron a numero 1
songsWRatingN1 = dfAll
songsWRatingN1 = songsWRatingN1.sort_values(['rank', 'date'], ascending = True)
songsWRatingN1 = songsWRatingN1.groupby(['canción']).agg({'rank': 'first', 
'release_date' : 'first','date': 'first', 'year' : 'first'}).reset_index() 
songsWRatingN1 = songsWRatingN1.sort_values(['date', 'rank'], ascending = True)



##Todas las canciones, primera aparición

songsWRatingFA = dfAll
songsWRatingFA = songsWRatingFA.sort_values(by='date', ascending = True).groupby(['canción']).agg({'rank': 'min', 
'release_date' : 'first', 'peak_rank' : 'min', 
'date': 'first', 'year' : 'first'}).reset_index() 



In [50]:
##2010 - cantidad de dias entre peak y lanzamiento
dfPeak2010sN1 = songsWRatingN1
dfPeak2010sN1 = dfPeak2010sN1[dfPeak2010sN1['year'] == 2010]
dfPeak2010sN1['dias_entre_lanzamiento_y_peak'] = (dfPeak2010sN1['date'] - dfPeak2010sN1['release_date']).dt.days.abs()



In [51]:
##2010 - canciones 2010s primera aparición en billboard
df2010sFA = songsWRatingFA[songsWRatingFA['year'] == 2010]
df2010sFA['dias_entre_lanzamiento_y_aparición'] = (df2010sFA['date'] - df2010sFA['release_date']).dt.days.abs()


In [52]:
##2020 - canciones 2020s numero 1 dias entre peak y lanzamiento
dfPeak2020sN1 = songsWRatingN1[songsWRatingN1['year'] == 2020]
dfPeak2020sN1['dias_entre_lanzamiento_y_peak'] = (dfPeak2020sN1['date'] - dfPeak2020sN1['release_date']).dt.days.abs()



In [53]:
##2020 - primera aparición canciones en billboard, dias entre primera apariciín y lanzamiento
df2020sFA = songsWRatingFA[songsWRatingFA['year'] == 2020]
df2020sFA['dias_entre_lanzamiento_y_aparición'] = (df2020sFA['date'] - df2020sFA['release_date']).dt.days.abs()


In [54]:
#Unir primera aparición con peak
faVsPeak = songsWRatingN1.merge(songsWRatingFA, how='left', on=['canción','year'])
faVsPeak['dias_entre_peak_y_aparición'] = (faVsPeak['date_x'] - faVsPeak['date_y']).dt.days.abs()


#gráfico mostrando dias entre priemera aparicion vs peak 2010
relvsPeak = px.scatter(faVsPeak[faVsPeak['year'] == 2010], x='date_x', y="date_y", color='canción', 
title='Distribución, relación de días entre primera aparición y pico en Billboard, 2010',
labels = {'date_y' : 'Primera aparición', 'date_x' : 'Primera fecha n° 1 en Billboard', 'canción' : 'Canción'} )


relvsPeak.show()

#gráfico mostrando dias entre priemera aparicion vs peak 2020
relvsPeak2020 = px.scatter(faVsPeak[faVsPeak['year'] == 2020], x='date_x', y="date_y", color='canción', 
title='Distribución, relación de días entre primera aparición y pico en Billboard, 2020',
labels = {'date_y' : 'Primera aparición', 'date_x' : 'Primera fecha n° 1 en Billboard', 'canción' : 'Canción'} )

relvsPeak2020.show()

#gráfico mostrando dias entre priemera aparicion ambos años
faVsPeakAverageComp = px.box( faVsPeak, x= 'year',
y = 'dias_entre_peak_y_aparición', points='all', color = 'year',
 title='Distribución, días entre primera aparición en Billboard y pico')


faVsPeakAverageComp.show()



In [55]:
##Crear data frame con el conteo manual de géneros en el n1, 2010
df = pd.DataFrame(dict(
    r=[12, 10, 6, 5, 2, 1],
    theta=['Pop','EDM','R&B',
           'Hip-hop', 'Rock', 'Reggae']))

##grafico radial mostrando los generos 2010          
figRad = px.line_polar(df, r='r', theta='theta', line_close=True)
figRad.update_traces(fill='toself')
figRad.show()

##Crear data frame con el conteo manual de géneros en el n1, 2020
df2 = pd.DataFrame(dict(
    r=[12, 6, 7, 12, 4, 1, 1, 2],
    theta=['Pop','EDM','R&B',
           'Hip-hop', 'Rock', 'Reggae', 'Christmas', 'Folk']))
##grafico radial mostrando los generos  2020  
figRad2 = px.line_polar(df2, r='r', theta='theta', line_close=True)
figRad2.update_traces(fill='toself')
figRad2.show()
